In [5]:
dayObs = '20250420'
run = "None"
sensor='R10_S10'
guidersensor='R44_SG0'

In [6]:
from lsst.daf.butler import Butler
from astropy.table import Table
from astropy.time import Time, TimeDelta
import tabulate
import numpy

In [7]:
if dayObs=='Today':
    dayObs = (Time.now() - TimeDelta(0.5, format='jd')).iso[:10].replace('-','')

In [9]:
butler = Butler('embargo_new', collections=[
    'LSSTCam/raw/all',
#    'LSSTCam/calib/unbounded',
#    'LSSTCam/photodiode'
])

OperationalError: (psycopg2.OperationalError) connection to server at "usdf-butler-embargo-db-tx.sdf.slac.stanford.edu" (172.24.10.17), port 5432 failed: fe_sendauth: no password supplied

(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [5]:
exposureList = []

if run == "None":
    where=f"exposure.day_obs={dayObs} and instrument='LSSTCam'"
else:
    where=f"exposure.science_program='{run}' and instrument='LSSTCam'"    

for record in butler.registry.queryDimensionRecords("exposure", where=where):
    nimage=len(list(butler.registry.queryDatasets('raw',dataId=record.dataId)))
    guider=len(list(butler.registry.queryDatasets('photodiode',dataId=record.dataId)))
    exposureList.append([record.id, record, nimage, guider])
exposureList.sort(key=lambda x: x[0])
table = Table(rows=[ (
            f'<a href=http://lsstcam-mcm.cp.lsst.org/FITSInfo/view.html?image={record.obs_id} target="_blank">{record.obs_id}</a>',
            record.timespan.begin.iso,
            record.seq_num,
            record.observation_type,
            record.science_program,
            record.observation_reason,
            record.exposure_time,
            record.physical_filter,
            nimage,
            f'<a href=https://usdf-rsp.slac.stanford.edu/times-square/github/lsst/CameraTimesSquare/Run7/GuiderData?expid={record.obs_id}&sensor={guidersensor}&ts_hide_code=1 target="_blank">Guider</a>',
            f'<a href=https://usdf-rsp.slac.stanford.edu/times-square/github/lsst/CameraTimesSquare/Run7/FitsHeader?expid={record.obs_id}&detector={sensor} target="_blank" >meta</a>',    
                     )
            for [id,record,nimage, guider] in exposureList ]
            )

In [6]:
subtable=Table(rows=[  ( record.obs_id, record.seq_num, nimage ) for [id,record,nimage, guider] in exposureList ], names=(  'id', 'seqnum', 'nimage'  ) )

# Possible data transfer issue

In [7]:
for a, b in zip(subtable[:-1][numpy.diff(subtable['seqnum'])!=1], subtable[1:][numpy.diff(subtable['seqnum'])!=1]):
    print(a['id'], b['id'])

# Changes in number of images ingested

In [8]:
for a, b in zip(subtable[:-1][numpy.diff(subtable['nimage'])!=0], subtable[1:][numpy.diff(subtable['nimage'])!=0]):
    print(a['id'], a['nimage'], b['id'], b['nimage'])

# Full records

In [9]:
print(f"The number of exposures is {len(table)}")
tabulate.tabulate(table[::-1], tablefmt='unsafehtml',headers=[
                   'obs_id', 'time', 'seqnum', 'observation_type',
                   'science_program', 'observation_reason','exposure_time',
                   'physical_filter', 'nimage', 'Guider', 'metadata'
                  ])

The number of exposures is 694


obs_id,time,seqnum,observation_type,science_program,observation_reason,exposure_time,physical_filter,nimage,Guider,metadata
MC_O_20250420_000694,2025-04-21 11:57:11.412502,694,bias,BLOCK-T454,block-t454,0,i_39,197,Guider,meta
MC_O_20250420_000693,2025-04-21 11:47:08.340939,693,bias,BLOCK-T454,block-t454,0,i_39,197,Guider,meta
MC_O_20250420_000692,2025-04-21 11:37:05.461248,692,bias,BLOCK-T454,block-t454,0,i_39,197,Guider,meta
MC_O_20250420_000691,2025-04-21 11:27:02.542611,691,bias,BLOCK-T454,block-t454,0,i_39,197,Guider,meta
MC_O_20250420_000690,2025-04-21 11:16:59.673099,690,bias,BLOCK-T454,block-t454,0,i_39,197,Guider,meta
MC_O_20250420_000689,2025-04-21 11:06:56.676490,689,bias,BLOCK-T454,block-t454,0,i_39,197,Guider,meta
MC_O_20250420_000688,2025-04-21 10:56:53.671835,688,bias,BLOCK-T454,block-t454,0,i_39,197,Guider,meta
MC_O_20250420_000687,2025-04-21 10:46:50.768808,687,bias,BLOCK-T454,block-t454,0,i_39,197,Guider,meta
MC_O_20250420_000686,2025-04-21 10:36:47.743151,686,bias,BLOCK-T454,block-t454,0,i_39,197,Guider,meta
MC_O_20250420_000685,2025-04-21 09:06:16.671584,685,acq,Test_images,test_images,15,i_39,197,Guider,meta
